In [ ]:
import pandas as pd
import os
import subprocess
import re
import shutil

In [ ]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy', #libx264 to encode if needed
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [ ]:
def get_columns_to_cut(df):
    pattern = re.compile(r'^rausschneiden\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [ ]:
def calculate_time(*timestamps,kind="addition"):
    """
    Calculate time based on the given timestamps and operation type.

    Parameters:
    - timestamps (str): Variable number of timestamps.
    - kind (str): The operation type. Allowed values are 'addition' and 'subtraction'.
    """
    
    #addition
    if kind == "addition":
        stamp1 = None
        for timestamp in timestamps:
            if timestamp == "nan":
                timestamp = "00:00:00"
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 + pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp
    
    #subtraction
    elif kind == "subtraction":
        stamp1 = None
        for timestamp in timestamps:
            if timestamp == "nan":
                timestamp = "00:00:00"        

        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 - pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp  

In [ ]:
def calculate_timestamp(input_timestamp,row_number):
    columns_to_cut = get_columns_to_cut(video_schnitt_df)
    pattern = re.compile(r'^rausschneiden\d+_bis$')
    result = "00:00:00"

    for col_num,col in enumerate(video_schnitt_df[columns_to_cut]):
        
        ab_timestamp = video_schnitt_df[columns_to_cut].iloc[row_number, col_num-1]
        bis_timestamp = video_schnitt_df[col][row_number]

        # check wether the timestamp in 'rausschneidenX_bis' is smaller than input
        if pattern.match(col):
            if bis_timestamp != "nan" and bis_timestamp < input_timestamp:
                # print(video_name, "col name:", col)
                # print("bis timestamp:", bis_timestamp, "ist kleiner als input Tiestamp:",input_timestamp)
                # print("ab Timestamp ist:", ab_timestamp)
                # print("bis Timestamp:",bis_timestamp,"minus",ab_timestamp)
                diff = calculate_time(bis_timestamp,ab_timestamp,kind="subtraction")
                result = calculate_time(result,diff,kind="addition")
                # print("Differenz:",diff,"Summe:",result)
                
    new_timestamp = calculate_time(input_timestamp,result,kind="subtraction")
    # print("neuer Teimestamp:",new_timestamp)
    result = "00:00:00"

    return new_timestamp

In [ ]:
def cut_in_2_pieces(input_file, timestamp, head_output, tail_output):
    
    # -ss = ab dort; -to = bis dort
    head_cmd = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c:v', 'copy', '-c:a', 'copy', head_output] # "-c:v" -> "libx264" to encrypt or "copy" to test
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c:v', 'copy', '-c:a', 'copy', tail_output] # "-c:v" -> "libx264" to encrypt or "copy" to test

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [ ]:
def get_columns_to_pause(df):
    pattern = re.compile(r'^pause\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [ ]:
def create_silent_audio(duration, output_file):
    # Command to create silent audio with ffmpeg
    command = [
        'ffmpeg',
        '-f', 'lavfi',
        '-i', 'anullsrc=channel_layout=stereo:sample_rate=48000',
        '-t', duration,
        '-c:a', 'aac',
        '-b:a', '191k',
        output_file
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Silent audio file {output_file} created successfully.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error creating silent audio file: {e}")

In [ ]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'libx264', # needs to stay libx264
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [ ]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [ ]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get file and clean it

In [ ]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)             

In [ ]:
video_schnitt_df

# Create pauses

In [ ]:
columns_to_pause = get_columns_to_pause(video_schnitt_df)


In [ ]:
# get stanbilder
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "dauer" in col:
            continue
        output = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        timestamp = video_schnitt_df[col][row]

        try:
            print(video_name, "standbild bei:", timestamp, "output:", output)
            get_standbild(video_name,output,timestamp)
        except Exception as e:
            print(e)
        
        counter += 1

In [ ]:
# get audios with silence for duration
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        output = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        timestamp = video_schnitt_df[col][row]
        
        try:
            print(video_name, "silence for:", timestamp, "output:", output)
            create_silent_audio(timestamp,output)
        except Exception as e:
            print(e)

        counter += 1

In [ ]:
# merge all the files
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        video_file = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        audio_file = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        output_file = f"{video_name.split('.')[0]}_pause{counter}.mp4"
        try :
            print(video_name,audio_file,output_file)
            merge_audio_and_video(video_file,audio_file,output_file)
        except Exception as e:
            print(e)
            
        counter += 1

# Handle output

In [ ]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

In [ ]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [ ]:
# move pauses
try:
    for video_name in video_schnitt_df["dateiname"]:
        original_prefix = video_name.split('.')[0]
        pattern = re.compile(rf'^{re.escape(original_prefix)}_pause\d+.*$')
        
        for f in os.listdir():
            if pattern.match(f):
                print(f)
                shutil.move(f,path_folder_endprodukte)
            #elif f.endswith(".mp4") and f not in original_video_names:
                #os.remove(f)

except Exception as e:
    print(e, "or this file already has been removed")

In [ ]:
# delete byproducts
try:
    for video_name in video_schnitt_df["dateiname"]:
        
        for f in os.listdir():
            if f.endswith(".mp4") and f not in original_video_names:
                os.remove(f)

except Exception as e:
    print(e, "or this file already has been removed")

## Cut_and_keep videos at specified pause-timestamps

In [ ]:
folder_path = "endprodukte/"
versions_to_concat = {}

try:
    for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
        counter = 1
        columns_to_pause = get_columns_to_pause(video_schnitt_df)
        for col_num, col in enumerate(video_schnitt_df[columns_to_pause]):
            
            if "bei" in col and video_schnitt_df[col][row] != "nan":
                
                # Start bei mindestens einer Teilung
                if counter == 1:

                    # define all variables needed
                    if os.path.exists(video_name):
                        input_file = video_name
                    else:
                        input_file = folder_path + video_name

                    # timestamp                
                    timestamp = video_schnitt_df[col][row]
                    # print("input timestamp:",timestamp)
                    timestamp = calculate_timestamp(timestamp,row)
                    # print("output timestamp:",timestamp)
                    
                    head_output = f"{video_name.split('.')[0]}_head{counter}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{counter}.mp4"
                    
                    print("all variables:",input_file,timestamp,head_output,tail_output)

                    #start separation
                    cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                    
                    versions_to_concat[video_name] = [head_output,f"{folder_path}{video_name.split('.')[0]}_pause{counter}.mp4",tail_output]
                    
                    counter += 1
                
                else:

                    # define alle the vars
                    input_file = f"{video_name.split('.')[0]}_tail{counter-1}.mp4"

                    # timestamp
                    timestamp1 = video_schnitt_df[columns_to_pause[col_num -2]][row]
                    timestamp1 = calculate_timestamp(timestamp1,row)

                    timestamp2 = video_schnitt_df[col][row]
                    timestamp2 = calculate_timestamp(timestamp2,row)

                    timestamp = calculate_time(timestamp2,timestamp1,kind="subtraction")


                    head_output = f"{video_name.split('.')[0]}_mitte{counter-1}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{counter}.mp4"
                    print("PRINT: timestamps before calculation:",timestamp1,timestamp2)
                    print("PRINT: all Variables further cut:",input_file,timestamp,head_output,tail_output)

                    cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                    os.remove(input_file)

                    versions_to_concat[video_name].pop()
                    versions_to_concat[video_name].append(head_output)
                    versions_to_concat[video_name].append(f"{folder_path}{video_name.split('.')[0]}_pause{counter}.mp4")
                    versions_to_concat[video_name].append(tail_output)

                    counter += 1
        
            
except Exception as e:
    print("Error in insert_pause part:", e)

## Concat pauses with the videosnippets

In [ ]:
try:
    for key, values in versions_to_concat.items():
        textfile_content = ""
        output = f"{key.split('.')[0]}_endprodukt.mp4"

        for value in values:
            dummy_string = f"file '{value}'\n"
            textfile_content = textfile_content + dummy_string

        # Create textfile with content
        textfile_name = "dummy.txt"
        with open(textfile_name, 'w') as textfile:
            textfile.write(textfile_content.rstrip())
            
        # Concat the videos
        print("key:",key,"ouput:",output,"textfile_content:", textfile_content)
        concatenate_videos(textfile_name, output)

except Exception as e:
    print(e)

In [ ]:
# delete byproducts
for key, values in versions_to_concat.items():

    for value in values:
        os.remove(value)